##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 독립적인 슬롯머신 손잡이를 가진 다중 슬롯머신에 대한 튜토리얼

### 시작하기

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/per_arm_bandits_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/agents/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/agents/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/agents/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

이 튜토리얼은 상황별 슬롯머신 문제에 TF-Agents를 사용하는 방법을 단계별로 안내합니다. 여기서 행동(손잡이)에는 특성(장르, 출시 연도...)에 따라 표시되는 영화 목록과 같은 고유한 특성을 가집니다.

### 전제 조건

여기에서 독자는 이 튜토리얼을 읽기 전에 [TF-Agents의 슬롯머신 튜토리얼](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb)을 통해 작업한 경험이 있으며, TF-Agents의 슬롯머신 라이브러리에 다소 익숙하다고 가정합니다.

## 손잡이 특성을 가진 다중 슬롯머신

"기본적인" 상황별 다중 슬롯머신 설정에서 에이전트는 매 시간 단계마다 컨텍스트 벡터(관찰이라고도 함)를 수신하며, 누적 보상을 최대화하기 위해 유한한 수량의 행동 세트(손잡이) 중에서 선택해야 합니다.

이제 에이전트가 사용자에게 다음에 시청할 영화를 추천하는 시나리오를 생각해봅니다. 결정을 내려야 할 때마다 에이전트는 사용자에 대한 일부 정보(시청 이력, 장르 선호도 등)와 선택 대상 영화 목록을 컨텍스트로 수신합니다.

우리는 사용자 정보를 컨텍스트로 사용하고 손잡이가 `movie_1, movie_2, ..., movie_K`가 되도록 함으로써 이 문제를 공식화할 수 있지만 이러한 접근 방식에서는 다음과 같은 여러 단점이 있습니다.

- 행동의 수는 시스템 내의 영화 수량으로 한정되며 새 영화를 추가하는 것은 번거롭습니다.
- 에이전트는 모든 영화에 대한 모델을 개별적으로 학습해야 합니다.
- 영화 간의 유사성은 고려되지 않습니다.

영화에 숫자를 부여하는 대신 더 직관적인 작업을 수행할 수 있습니다. 즉, 장르, 상영 시간, 출연진, 시청 등급, 출시 연도 등의 특성 세트로 영화를 표시할 수 있습니다. 이러한 접근 방식은 다음과 같은 장점을 가집니다.

- 영화 전반에 걸친 일반화가 가능합니다.
- 에이전트는 사용자 및 영화 특성으로 보상을 모델링하는 하나의 보상 함수만 학습합니다.
- 시스템에서 제거하거나 시스템에 새 영화를 추가하는 작업이 간편합니다.

이 새로운 설정에서는 행동의 수가 매 시간 단계마다 동일할 필요도 없습니다.


## TF-Agents의 독립적인 손잡이를 가진 슬롯머신

TF-Agents 슬롯머신 도구 모음은 독립적인 손잡이를 가진 슬롯머신에서도 사용할 수 있도록 개발되었습니다. 독립적인 손잡이 환경이 있으며 대부분의 정책 및 에이전트는 독립적인 손잡이 모드로 작동할 수 있습니다.

예시 코딩을 시작하기 전에 필수적으로 가져와야 하는 요소가 있습니다.

### 설치

In [ ]:
!pip install tf-agents

### 가져오기 항목

In [ ]:
import functools
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.environments import stationary_stochastic_per_arm_py_environment as p_a_env
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts

nest = tf.nest

### 매개변수 -- 자유롭게 둘러보기

In [ ]:
# The dimension of the global features.
GLOBAL_DIM = 40  #@param {type:"integer"}
# The elements of the global feature will be integers in [-GLOBAL_BOUND, GLOBAL_BOUND).
GLOBAL_BOUND = 10  #@param {type:"integer"}
# The dimension of the per-arm features.
PER_ARM_DIM = 50  #@param {type:"integer"}
# The elements of the PER-ARM feature will be integers in [-PER_ARM_BOUND, PER_ARM_BOUND).
PER_ARM_BOUND = 6  #@param {type:"integer"}
# The variance of the Gaussian distribution that generates the rewards.
VARIANCE = 100.0  #@param {type: "number"}
# The elements of the linear reward parameter will be integers in [-PARAM_BOUND, PARAM_BOUND).
PARAM_BOUND = 10  #@param {type: "integer"}

NUM_ACTIONS = 70  #@param {type:"integer"}
BATCH_SIZE = 20  #@param {type:"integer"}

# Parameter for linear reward function acting on the
# concatenation of global and per-arm features.
reward_param = list(np.random.randint(
      -PARAM_BOUND, PARAM_BOUND, [GLOBAL_DIM + PER_ARM_DIM]))

### 간단한 독립적인 손잡이 환경

다른 [튜토리얼](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb)에서 설명한 정상확률 환경에는 독립적인 손잡이와 비교되는 환경이 있습니다.

독립적인 손잡이 환경을 초기화하려면 다음을 생성하는 함수를 정의해야 합니다.

- *전역 및 독립적인 손잡이 특성*: 이러한 함수에는 입력 매개변수가 없으며 호출될 때 단일(전역 또는 독립적인 손잡이) 특성 벡터를 생성합니다.
- *보상*: 이 함수는 전역 및 독립적인 손잡이 특성 벡터의 연결을 매개변수로 사용하고, 보상을 생성합니다. 기본적으로 이것은 에이전트가 "추측"해야 하는 함수입니다. 독립적인 손잡이 사례에서 모든 손잡이에 대해 보상 함수가 동일하다는 점은 주목할 만한 가치가 있습니다. 이것은 에이전트가 각 손잡이에 대한 보상 함수를 독립적으로 추정해야 하는 일반적인 슬롯머신 사례와는 근본적으로 다릅니다.


In [ ]:
def global_context_sampling_fn():
  """This function generates a single global observation vector."""
  return np.random.randint(
      -GLOBAL_BOUND, GLOBAL_BOUND, [GLOBAL_DIM]).astype(np.float32)

def per_arm_context_sampling_fn():
  """"This function generates a single per-arm observation vector."""
  return np.random.randint(
      -PER_ARM_BOUND, PER_ARM_BOUND, [PER_ARM_DIM]).astype(np.float32)

def linear_normal_reward_fn(x):
  """This function generates a reward from the concatenated global and per-arm observations."""
  mu = np.dot(x, reward_param)
  return np.random.normal(mu, VARIANCE)

이제 환경을 초기화할 수 있습니다.

In [ ]:
per_arm_py_env = p_a_env.StationaryStochasticPerArmPyEnvironment(
    global_context_sampling_fn,
    per_arm_context_sampling_fn,
    NUM_ACTIONS,
    linear_normal_reward_fn,
    batch_size=BATCH_SIZE
)
per_arm_tf_env = tf_py_environment.TFPyEnvironment(per_arm_py_env)

아래에서 이 환경의 결과물을 확인할 수 있습니다.

In [ ]:
print('observation spec: ', per_arm_tf_env.observation_spec())
print('\nAn observation: ', per_arm_tf_env.reset().observation)

action = tf.zeros(BATCH_SIZE, dtype=tf.int32)
time_step = per_arm_tf_env.step(action)
print('\nRewards after taking an action: ', time_step.reward)

우리는 관찰 사양이 두 개의 요소가 있는 사전임을 알 수 있습니다.

- 키가 `'global'`인 항목: 이것은 `GLOBAL_DIM` 매개변수와 일치하는 형상이 있는 전역 컨텍스트 부분입니다.
- 키가 `'per_arm'`인 항목: 이것은 독립적인 손잡이의 컨텍스트이며 형상은 `[NUM_ACTIONS, PER_ARM_DIM]`입니다. 이것은 시간 단계의 모든 손잡이에 대한 손잡이 특성에 해당하는 자리 표시자 부분입니다.


### LinUCB 에이전트

LinUCB 에이전트는 선형 보상 함수의 매개변수를 추정하는 동시에 추정치에 대한 신뢰 타원체를 유지하는 동일한 이름의 슬롯머신 알고리즘을 구현합니다. 에이전트는 매개변수가 신뢰 타원체 내에 있다고 가정하며 예상 보상이 가장 높은 손잡이를 선택합니다.

에이전트를 생성하려면 관찰 및 행동 사양에 대한 지식이 필요합니다. 에이전트를 정의할 경우 부울 매개변수 `accepts_per_arm_features`를 `True`로 설정합니다.

In [ ]:
observation_spec = per_arm_tf_env.observation_spec()
time_step_spec = ts.time_step_spec(observation_spec)
action_spec = tensor_spec.BoundedTensorSpec(
    dtype=tf.int32, shape=(), minimum=0, maximum=NUM_ACTIONS - 1)

agent = lin_ucb_agent.LinearUCBAgent(time_step_spec=time_step_spec,
                                     action_spec=action_spec,
                                     accepts_per_arm_features=True)

### 훈련 데이터의 흐름

이 섹션에서는 독립적인 손잡이 특성이 정책에서 훈련으로 이동하는 방법에 대한 역할을 맛보기 정도로만 설명합니다. 다음 섹션(후회 메트릭 정의하기)으로 자유롭게 이동하였다가 나중에 관심이 생겼을 때 여기로 다시 돌아오세요.

먼저 에이전트의 데이터 사양을 살펴봅니다. 에이전트의 `training_data_spec` 속성은 훈련 데이터가 가져야 하는 요소와 구조를 지정합니다.

In [ ]:
print('training data spec: ', agent.training_data_spec)

사양의 `observation` 부분을 자세히 살펴보면 독립적인 손잡이 특성이 포함되어 있지 않음을 알 수 있습니다!

In [ ]:
print('observation spec in training: ', agent.training_data_spec.observation)

독립적인 손잡이 특성은 어떻게 되었을까요? 이 질문에 답하려면 먼저 LinUCB 에이전트가 학습할 때  **모든** 손잡이의 독립적인 손잡이 특성을 필요로 하지 않고 **선택한** 손잡이의 특성만 필요로 한다는 점을 주목해야 합니다. 따라서 `[BATCH_SIZE, NUM_ACTIONS, PER_ARM_DIM]` 형상의 텐서는 특히 행동의 수가 많은 경우 매우 불필요해질 수 있기 때문에 제외하는 것이 이치에 맞습니다.

그러나 여전히 선택한 손잡이의 독립적인 손잡이 특성은 어딘가에 있어야 합니다! 이를 위하여 LinUCB 정책이 훈련 데이터의 `policy_info` 필드에 선택한 손잡이의 특성을 저장하는지 확인해야 합니다.

In [ ]:
print('chosen arm features: ', agent.training_data_spec.policy_info.chosen_arm_features)

우리는 `chosen_arm_features` 필드에 한 손잡이의 특성 벡터만 있고 이것이 선택된 손잡이가 될 것임을 형상을 통해 알 수 있습니다. `policy_info`와 `chosen_arm_features`는 훈련 데이터 사양 검사에서 보았듯이 훈련 데이터의 일부이므로 훈련 시 사용할 수 있습니다.

### 후회 메트릭 정의하기

훈련 루프를 시작하기 전에 에이전트의 후회를 계산하는 데 도움이 되는 몇 개의 유틸리티 함수를 정의합니다. 이러한 함수는 일련의 행동(손잡이 특성으로 제공)과 에이전트에게 숨겨진 선형 매개변수가 제공될 경우 최적의 예상 보상을 결정하는 데 도움이 됩니다.

In [ ]:
def _all_rewards(observation, hidden_param):
  """Outputs rewards for all actions, given an observation."""
  hidden_param = tf.cast(hidden_param, dtype=tf.float32)
  global_obs = observation['global']
  per_arm_obs = observation['per_arm']
  num_actions = tf.shape(per_arm_obs)[1]
  tiled_global = tf.tile(
      tf.expand_dims(global_obs, axis=1), [1, num_actions, 1])
  concatenated = tf.concat([tiled_global, per_arm_obs], axis=-1)
  rewards = tf.linalg.matvec(concatenated, hidden_param)
  return rewards

def optimal_reward(observation):
  """Outputs the maximum expected reward for every element in the batch."""
  return tf.reduce_max(_all_rewards(observation, reward_param), axis=1)

regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward)

이제 슬롯머신 훈련 루프를 시작할 준비가 완료되었습니다. 아래의 드라이버는 정책을 사용하여 행동을 선택하고, 선택한 행동의 보상을 재생 버퍼에 저장하고, 사전에 정의한 후회 메트릭을 계산하고, 에이전트의 훈련 단계를 실행하는 작업을 처리합니다.

In [ ]:
num_iterations = 20 # @param
steps_per_loop = 1 # @param

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.policy.trajectory_spec,
    batch_size=BATCH_SIZE,
    max_length=steps_per_loop)

observers = [replay_buffer.add_batch, regret_metric]

driver = dynamic_step_driver.DynamicStepDriver(
    env=per_arm_tf_env,
    policy=agent.collect_policy,
    num_steps=steps_per_loop * BATCH_SIZE,
    observers=observers)

regret_values = []

for _ in range(num_iterations):
  driver.run()
  loss_info = agent.train(replay_buffer.gather_all())
  replay_buffer.clear()
  regret_values.append(regret_metric.result())


이제 결과를 확인합니다. 작업을 올바르게 수행하였다면 에이전트는 선형 보상 함수를 잘 추정할 수 있으며 그 결과 정책은 예상 보상이 최적의 보상에 가까운 행동을 선택할 수 있습니다. 이는 위에서 정의한 후회 메트릭으로 표시되며, 이 값은 점점 내려가서 0에 근접합니다.

In [ ]:
plt.plot(regret_values)
plt.title('Regret of LinUCB on the Linear per-arm environment')
plt.xlabel('Number of Iterations')
_ = plt.ylabel('Average Regret')

### 다음은?

위의 예시는 [뉴럴 입실론-그리디 에이전트](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/neural_epsilon_greedy_agent.py)를 비롯한 다른 에이전트에서도 선택할 수 있는 코드베이스에서 [구현](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/examples/v2/train_eval_per_arm_stationary_linear.py)되었습니다.